In [5]:
from deep_logic import extract_logic

# Example text
text = "If it rains, the ground gets wet. It is raining. Therefore, the ground is wet."

# Extract logic
logic_output = extract_logic(text)

# Print the extracted logic
print(logic_output)

ImportError: cannot import name 'extract_logic' from 'deep_logic' (/Users/mac/Documents/630/Project/Logical_Classification/.venv/lib/python3.11/site-packages/deep_logic/__init__.py)

In [6]:
import torch
import numpy as np
import deep_logic as dl

In [10]:
np.random.seed(0)   
torch.manual_seed(0)  

In [11]:
x_train = torch.tensor([
    [0, 0, 0, 1],
    [0, 1, 0, 1],
    [1, 0, 0, 1],
    [1, 1, 0, 1],
], dtype=torch.float)
y_train = torch.tensor([0, 1, 1, 0], dtype=torch.float).unsqueeze(1)
xnp = x_train.detach().numpy()
ynp = y_train.detach().numpy().ravel()

In [12]:
layers = [
    torch.nn.Linear(x_train.size(1), 10),
    torch.nn.LeakyReLU(),
    torch.nn.Linear(10, 4),
    torch.nn.LeakyReLU(),
    torch.nn.Linear(4, 1),
    torch.nn.Sigmoid(),
]
model = torch.nn.Sequential(*layers)

In [13]:
dl.validate_data(x_train)

AttributeError: module 'deep_logic' has no attribute 'validate_data'

In [15]:
print(dir(dl))

['__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', '_version', 'concept_extractor', 'logic', 'models', 'nn', 'utils']


In [18]:
print(dir(dl.nn))

['Conv2Concepts', 'XLinear', 'XLogic', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'concepts', 'linear', 'logic']


In [19]:
print(dir(dl.utils))

['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'base', 'loss', 'metrics', 'relu_nn', 'selection']


In [23]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
model.train()
need_pruning = True
for epoch in range(1000):
    # forward pass
    optimizer.zero_grad()
    y_pred = model(x_train)

    # Compute Loss
    loss = torch.nn.functional.binary_cross_entropy(y_pred, y_train)
    # A bit of L1 regularization will encourage sparsity
    for module in model.children():
        if isinstance(module, torch.nn.Linear):
            loss += 0.001 * torch.norm(module.weight, 1)

    # We can use sparsity to prune dummy features
    if epoch > 500 and need_pruning:
        # dl.utils.relu_nn.prune_features(model, n_classes)
        dl.utils.relu_nn.prune_features(model, n_classes=2)
        need_pruning = False


    # backward pass
    loss.backward()
    optimizer.step()

    # compute accuracy
    if epoch % 100 == 0:
        y_pred_d = (y_pred > 0.5)
        accuracy = (y_pred_d.eq(y_train).sum(dim=1) == y_train.size(1)).sum().item() / y_train.size(0)
        print(f'Epoch {epoch}: train accuracy: {accuracy:.4f}')

Epoch 0: train accuracy: 1.0000
Epoch 100: train accuracy: 1.0000
Epoch 200: train accuracy: 1.0000
Epoch 300: train accuracy: 1.0000
Epoch 400: train accuracy: 1.0000
Epoch 500: train accuracy: 1.0000
Pruned 0/4 features
Pruned 2/4 features
Epoch 600: train accuracy: 1.0000
Epoch 700: train accuracy: 1.0000
Epoch 800: train accuracy: 1.0000
Epoch 900: train accuracy: 1.0000


In [31]:
explanation = dl.logic.explain_local(model, x_train, y_train, x_sample=torch.tensor([x_train[1], y_train[1]]),
                                    #  x_sample=
                                     method='pruning', target_class=1,
                                     concept_names=['f1', 'f2', 'f3', 'f4'])
print(explanation)

ValueError: only one element tensors can be converted to Python scalars

In [32]:
explanation = dl.logic.explain_local(model, x_train, y_train, x_sample=x[1],
                                     method='pruning', target_class=1,
                                     concept_names=['f1', 'f2', 'f3', 'f4'])
print(explanation)

NameError: name 'x' is not defined

In [33]:
import lens
import torch

In [36]:
torch.manual_seed(0)
np.random.seed(0)

# lens.utils.base.set_seed(0)
x = torch.rand([100, 4])
y = (x[:, 0] > 0.5) & (x[:, 1] < 0.5) | \
    (x[:, 0] < 0.5) & (x[:, 1] > 0.5)

data = torch.utils.data.TensorDataset(x, y)

train_data, val_data, test_data = torch.utils.data.random_split(data, [80, 10, 10])
x_train, y_train = data[train_data.indices]
x_val, y_val = data[val_data.indices]
x_test, y_test = data[test_data.indices]

In [37]:
model = lens.models.XMuNN(n_classes=2, n_features=4,
                          hidden_neurons=[10], loss=torch.nn.CrossEntropyLoss())

AttributeError: module 'lens' has no attribute 'models'

In [ ]:
model.fit(train_data, val_data, epochs=100, l_r=0.1)

## get accuracy on test samples
test_acc = model.evaluate(test_data)
print("Test accuracy:", test_acc)

In [ ]:
## get first order logic explanations for a specific target class
target_class = 1
concept_names = ['x1', 'x2', 'x3', 'x4']
formula = model.get_global_explanation(x_train, y_train, target_class,
                                       top_k_explanations=2, concept_names=concept_names)
print(f"{formula} <-> f{target_class}")

## compute explanation accuracy
exp_accuracy, _ = lens.logic.test_explanation(formula, target_class, x_test, y_test,
                                              concept_names=concept_names)
print("Logic Test Accuracy:", exp_accuracy)